In [ ]:
#A projektemben lescrapeltem közel 8000 receptet nosaltyról, amiket aztán egy excel táblába gyűjtöttem.
#A projekt második felében a megformázott excel táblát felhasználva, írtam egy programot, ami hozzávalókat(inputokat)
#kér be, aztán olyan receptek linkjeit mutatja meg, amelyek receptjeiben minden megadott hozzávaló szerepel.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from csv import reader
import unicodecsv as csv
import xlsxwriter

In [9]:
#Receptek linkjeinek kigyűjtése

page0 = 'https://www.nosalty.hu'

receptlinkek = []#ebbe kerülnek majd a linkek
for x in range(1,2):#minden oldalon tizenöt link van, szóval a range 1-el növelése 15-el növeli a linkeket
    r = requests.get(f'https://www.nosalty.hu/receptek?page={x}')
    soup = BeautifulSoup(r.content, 'lxml')

    receptlista = soup.find_all('li', class_='m-grid__child')

    for item in receptlista:
        for link in item.find_all('a', href=True):
            receptlinkek.append(page0 + link['href'])
del receptlinkek[::2]#mivel a képbe és a címbe is be van ágyazva, ezért duplán szedte ki, ezért kitörlöm minden másodikat
print(receptlinkek)

['https://www.nosalty.hu/recept/olasz-cukros-fank', 'https://www.nosalty.hu/recept/csoben-sult-toltott-kaposzta', 'https://www.nosalty.hu/recept/repahummusz-grillgombaval', 'https://www.nosalty.hu/recept/klasszikus-francia-eclair-fank', 'https://www.nosalty.hu/recept/kremes-janokiandrea-konyhajabol', 'https://www.nosalty.hu/recept/grillezett-hal', 'https://www.nosalty.hu/recept/mezeskalacsos-muffin', 'https://www.nosalty.hu/recept/csirkefalatkak', 'https://www.nosalty.hu/recept/hazi-keszitesu-hasabburgonya', 'https://www.nosalty.hu/recept/egesz-grillcsirke', 'https://www.nosalty.hu/recept/sonkas-rakott-kelkaposzta-bulgurral', 'https://www.nosalty.hu/recept/hetkoznap-esti-minestrone', 'https://www.nosalty.hu/recept/eleszto-nelkuli-gyors-fank-gluten-laktoz-es-cukormentesen', 'https://www.nosalty.hu/recept/zoldsegchipsek-keksajtos-martogatossal', 'https://www.nosalty.hu/recept/azsiai-mazas-ragacsos-oldalas']


In [10]:
#ezekbe a listákba gyűjtöm az adatokat
kajalista = []
hozzavalolista = []
nagylista = []
for page in receptlinkek:#adatok kigyűjtése
    l = requests.get(page)
    soup = BeautifulSoup(l.content, 'html.parser')
    hozzavalok = soup.find(id='ingredients')
    kajanev = soup.find(class_='p-article__title -recipe pt-8 mb-5 d-block').text.strip()
    items = hozzavalok.find_all(class_='m-list__item p-2 -dotted -fontSize-16 d-flex justify-content-between pl-5')
    period_names = [item.find(class_='a-link -primaryHoverEffect -fontSize-16').get_text() for item in items]
    kajalista.append(kajanev)
    hozzavalolista.append(period_names)
striped = []
namostmartenyleg = []
receptjo = [''.join(ele) for ele in hozzavalolista]
for i in receptjo:
    striped.append(i.replace('\n', ''))
for i in striped:
    namostmartenyleg.append(i.replace('           ',' '))
    
print(kajalista)
print(namostmartenyleg)
print(receptlinkek)

#kigyűjtés excelbe
workbook = xlsxwriter.Workbook('receptkonyv.xlsx')
sheet = workbook.add_worksheet ()

sheet.write('A1', 'Étel neve')
sheet.write('B1', 'Receptek')
sheet.write('C1', 'Link')

#oszlopok meghatározása és kigyűjtése
for i in range(len(kajalista)):
    sheet.write(i+1, 0, kajalista[i])
for i in range(len(receptlinkek)):
    sheet.write(i+1, 1, receptlinkek[i])
for i in range(len(namostmartenyleg)):
    sheet.write(i+1, 2, namostmartenyleg[i])

workbook.close()

['Olasz cukros fánk', 'Töltött káposzta dagadóból', 'Répahummusz grillgombával', 'Klasszikus francia eclair fánk', 'Krémes janokiandrea konyhájából', 'Grillezett hal', 'Mézeskalácsos muffin', 'Csirkefalatkák', 'Házi készítésű hasábburgonya', 'Egész grillcsirke', 'Sonkás, rakott kelkáposzta bulgurral', 'Hétköznap esti minestrone', 'Élesztő nélküli, gyors fánk glutén-, laktóz- és cukormentesen', 'Zöldségchipsek kéksajtos mártogatóssal', 'Ázsiai stílusú mázas-ragacsos oldalas']
['    finomliszt    sütőpor    cukor    vanília kivonat    citromhéj    tojás    tej    napraforgó olaj          ', '    sertésdagadó    vöröshagyma    fokhagyma    füstölt szalonna    kolbász    császárszalonna    füstölt pirospaprika    őrölt fűszerkömény    só    bors    rizs    savanyú káposzta    káposztalevél    babérlevél    víz          ', '    csiperkegomba    sárgarépa    csicseriborsó konzerv    fokhagyma    koriander    tortilla lap    olívaolaj    só    bors    menta    víz          ', '    víz    vaj 